In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import re
import Levenshtein as lev
#import matplotlib.pyplot as plt

In [ ]:
##Create list of all structured files (from EPIC) and list with names of files
print('start0')
path_to_dir=r'Z:\final EPIC data 20220909\*.xlsx'
list_files=glob.glob(path_to_dir)
data_all=[]
file_name=[]
print('start1')
print(len(list_files))
for j in range(0,len(list_files)):
    print('j:', j)
    data_all.append(pd.read_excel(list_files[j]))
    file_name.append(list_files[j][32:-5])

print('start2')
##Read NKR registry data
NKR_data=pd.read_csv(r"Z:\IKNL data\IKNL data update\K21186.csv", delimiter=';')

print('start3')
##Read tables with mapping between Patient ID (from the structured files) to the pseudoID of the clinical notes
AMC_conv=pd.read_excel(r"Z:\final EPIC data 20220909\Text data 6-10-2022\002a AMC CTcue import-results-2022-09-12.xlsx")
VUmc_conv=pd.read_excel(r"Z:\final EPIC data 20220909\Text data 6-10-2022\002b VUmc CTcue import-results-2022-09-09.xlsx")

print('start4')
##Read clinical notes from location AMC and location VUmc
notes_AMC=pd.read_csv(r"Z:\final EPIC data 20220909\Text data 6-10-2022\amc_data_aanvraag_20221004_aangepast.csv", encoding="utf-16")
notes_VUmc=pd.read_csv(r"Z:\final EPIC data 20220909\Text data 6-10-2022\vumc_data_aanvraag_20221004_aangepast.csv", encoding="utf-16")

In [ ]:
notes_AMC['pseudonomised_text']=notes_AMC['pseudonomised_text'].str.replace('\r\n\r\n\r\n' ,'. ')
notes_AMC['pseudonomised_text']=notes_AMC['pseudonomised_text'].str.replace('\r\n\r\n','. ')
notes_AMC['pseudonomised_text']=notes_AMC['pseudonomised_text'].str.replace('\r\n',': ')
notes_VUmc['pseudonomised_text']=notes_VUmc['pseudonomised_text'].str.replace('\r\n\r\n\r\n' ,'. ')
notes_VUmc['pseudonomised_text']=notes_VUmc['pseudonomised_text'].str.replace('\r\n\r\n','. ')
notes_VUmc['pseudonomised_text']=notes_VUmc['pseudonomised_text'].str.replace('\r\n',': ')

In [ ]:
matches_VU = 0

# Define the fuzzy pattern with dynamic Levenshtein distance for individual words
pattern = r'(?i)\b(?:WHO|ECOG|Performance)\s*(?:PS|Status|Score)\b'

# Define a function to calculate the maximum Levenshtein distance for individual words
def max_levenshtein_distance(word):
    length = len(word)
    if length <= 5:
        return 1  # Allow small variations for short words
    else:
        return min(7, int(length * 0.4))  # Adjust the threshold for longer words

df = notes_VUmc   
    
# Replace non-breaking spaces with regular spaces
df['pseudonomised_text'] = df['pseudonomised_text'].str.replace(u'\u00a0', ' ')

# Iterate over the DataFrame and print the notes that match the pattern
for index, row in df.iterrows():
    text = row['pseudonomised_text']
    words = re.findall(r'\b\w+\b', text)  # Extract words for Levenshtein distance calculation
    matches = []
    for word in words:
        max_distance = max_levenshtein_distance(word)
        if any(lev.distance(word.lower(), term.lower()) <= max_distance for term in ['performance', 'status']):
            matches.append(word)
    if matches:
        #print(text)
        matches_VU +=1
        
print('total:', matches_VU)


In [ ]:
matches_AMC = 0

# Define the fuzzy pattern with dynamic Levenshtein distance for individual words
pattern = r'(?i)\b(?:WHO|ECOG|Performance)\s*(?:PS|Status|Score)\b'

# Define a function to calculate the maximum Levenshtein distance for individual words
def max_levenshtein_distance(word):
    length = len(word)
    if length <= 5:
        return 1  # Allow small variations for short words
    else:
        return min(7, int(length * 0.4))  # Adjust the threshold for longer words

df = notes_AMC   
    
# Replace non-breaking spaces with regular spaces
df['pseudonomised_text'] = df['pseudonomised_text'].str.replace(u'\u00a0', ' ')

# Iterate over the DataFrame and print the notes that match the pattern
for index, row in df.iterrows():
    text = row['pseudonomised_text']
    words = re.findall(r'\b\w+\b', text)  # Extract words for Levenshtein distance calculation
    matches = []
    for word in words:
        max_distance = max_levenshtein_distance(word)
        if any(lev.distance(word.lower(), term.lower()) <= max_distance for term in ['performance', 'status']):
            matches.append(word)
    if matches:
        #print(text)
        matches_AMC +=1
        
print('total:', matches_AMC)


In [ ]:
import re

def find_performance_scores(text):
  """
  Finds performance scores in a given text and extracts surrounding context.

  Args:
    text: The text to search.

  Returns:
    A list of dictionaries containing the score type, system (for ECOG/WHO),
    and the original surrounding context (if a match is found).
  """

  scores = []

  # Performance score (ECOG or WHO)
  pattern = r"(?P<before>.{0,50})((?:ECOG|WHO)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "performance",
      "system": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # Performance status (ECOG or WHO)
  pattern = r"(?P<before>.{0,50})((?:performance\s+status|status)\s*(?:ECOG|WHO)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "performance_status",
      "system": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # Karnofsky score
  pattern = r"(?P<before>.{0,50})((?:Karnofsky|KPS)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "karnofsky",
      "score": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # PS score
  pattern = r"(?P<before>.{0,50})((?:PS|P\.S\.|PS\.)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "ps",
      "score": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  return scores

def create_has_score_column(text):
  """
  Checks if there's a match in the text and returns the context if a match is found,
  otherwise returns an empty string.
  """
  scores = find_performance_scores(text)
  if scores:
    return scores[0]["context"]  # Assuming only the first match is needed
  else:
    return ""

# Add a new column named 'has_score' to store the context if a match is found
notes_VUmc['has_score'] = notes_VUmc['pseudonomised_text'].apply(create_has_score_column)

# Print the DataFrame to see the new 'has_score' column
print(notes_VUmc)


In [ ]:
notes_VUmc['has_score'].nunique()

In [ ]:
for text in notes_VUmc['has_score']:
    if len(text) > 1:
        print('')
        print(text)

In [ ]:
import re

def find_performance_scores(text):
  """
  Finds performance scores in a given text and extracts surrounding context.

  Args:
    text: The text to search.

  Returns:
    A list of dictionaries containing the score type, system (for ECOG/WHO),
    and the original surrounding context (if a match is found).
  """

  scores = []

  # Performance score (ECOG or WHO)
  pattern = r"(?P<before>.{0,50})((?:ECOG|WHO)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "performance",
      "system": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # Performance status (ECOG or WHO)
  pattern = r"(?P<before>.{0,50})((?:performance\s+status|status)\s*(?:ECOG|WHO)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "performance_status",
      "system": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # Karnofsky score
  pattern = r"(?P<before>.{0,50})((?:Karnofsky|KPS)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "karnofsky",
      "score": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  # PS score
  pattern = r"(?P<before>.{0,50})((?:PS|P\.S\.|PS\.)\s*(\d+))(?:(?P<after>.{0,50})|$)"
  for match in re.finditer(pattern, text):
    scores.append({
      "type": "ps",
      "score": match.group(3),
      "context": match.group("before") + match.group(0) + match.group("after")
    })

  return scores

def create_has_score_column(text):
  """
  Checks if there's a match in the text and returns the context if a match is found,
  otherwise returns an empty string.
  """
  scores = find_performance_scores(text)
  if scores:
    return scores[0]["context"]  # Assuming only the first match is needed
  else:
    return ""

# Add a new column named 'has_score' to store the context if a match is found
notes_AMC['has_score'] = notes_AMC['pseudonomised_text'].apply(create_has_score_column)

# Print the DataFrame to see the new 'has_score' column
print(notes_AMC)


In [ ]:
notes_AMC['has_score'].nunique()

In [ ]:
for text in notes_AMC['has_score']:
    if len(text) > 1:
        print('')
        print(text)

In [ ]:
notes_AMC['has_score']

In [ ]:
print(notes_AMC[notes_AMC['pseudonomised_text'].str.contains('PS')]['pseudonomised_text'])
pd.set_option('display.max_colwidth', None)